# Chapter 16: Multi-Agent Systems - Solutions
**From: Zero to AI Agent**

In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

---
## Exercise 16.1 Solution: Add a Reviewer Agent

In [ ]:
from typing import TypedDict, Annotated
from operator import add
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


class TeamState(TypedDict):
    task: str
    messages: Annotated[list[dict], add]
    next_agent: str
    work_done: str
    review: str
    approved: bool


def supervisor(state: TeamState) -> dict:
    messages = state.get("messages", [])
    approved = state.get("approved", False)
    
    if not messages:
        return {"next_agent": "worker"}
    elif not approved and len(messages) == 1:
        return {"next_agent": "reviewer"}
    else:
        return {"next_agent": "FINISH"}


def worker(state: TeamState) -> dict:
    task = state["task"]
    response = llm.invoke(f"Complete this task: {task}")
    return {
        "messages": [{"agent": "worker", "content": response.content}],
        "work_done": response.content
    }


def reviewer(state: TeamState) -> dict:
    work = state["work_done"]
    prompt = f"""Review this work and say APPROVED if good, or suggest improvements:
    {work}"""
    response = llm.invoke(prompt)
    approved = "APPROVED" in response.content.upper()
    return {
        "messages": [{"agent": "reviewer", "content": response.content}],
        "review": response.content,
        "approved": approved
    }


def route_to_agent(state: TeamState) -> str:
    return state["next_agent"]


# Build graph
graph = StateGraph(TeamState)

graph.add_node("supervisor", supervisor)
graph.add_node("worker", worker)
graph.add_node("reviewer", reviewer)

graph.set_entry_point("supervisor")

graph.add_conditional_edges(
    "supervisor",
    route_to_agent,
    {
        "worker": "worker",
        "reviewer": "reviewer",
        "FINISH": END
    }
)

graph.add_edge("worker", "supervisor")
graph.add_edge("reviewer", "supervisor")

team = graph.compile()

# Test
result = team.invoke({
    "task": "Write a haiku about coding",
    "messages": [],
    "next_agent": "",
    "work_done": "",
    "review": "",
    "approved": False
})

print(f"Work done: {result['work_done']}")
print(f"\nReview: {result['review']}")
print(f"\nApproved: {result['approved']}")

---
## Next Steps

Proceed to **Chapter 17: Advanced Patterns**